In [6]:
from multiprocessing.dummy import Pool as ThreadPool
import time
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen
import pandas as pd
import json
import os
    
## https://financialmodellingprep.com/developer/docs/company-enterprise-value-api

In [7]:
def get_jsonparsed_data(url):
    """
    Receive the content of \`\`url\`\`, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

def get_fin_data(Tickers):
    
    maindf = pd.DataFrame()

    for Ticker in Tickers:
        temp_co = pd.DataFrame()
        
        try:
            
            statements = ['income-statement', 
              'balance-sheet-statement', 
              'cash-flow-statement']

            for statement in statements:

                url = ("https://financialmodelingprep.com/api/v3/financials/" +str(statement) + "/" + str(Ticker))

                a = get_jsonparsed_data(url)

                df = pd.DataFrame.from_records(a.get('financials'))
                try:
                    df.date = df.date.apply(lambda x: str(x).replace('-','')).astype('int32')
                    df = df.loc[df['date']==int(df.date.max())]
                except:
                    pass
                df = df.T
                df[Ticker] = df[0]
                del df[0]
                df = pd.DataFrame(df.loc[df.index != 'date'])
                temp_co = temp_co.append(df)

            url = ("https://financialmodelingprep.com/api/v3/company/profile/" + str(Ticker))
            a = get_jsonparsed_data(url)
            df = pd.DataFrame.from_records(a)
            a = a.get('symbol')
            df[a] = df['profile']
            del df['profile']
            del df['symbol']

            temp_co = temp_co.append(df)

            url = ("https://financialmodelingprep.com/api/v3/enterprise-value/" + str(Ticker))
            a = get_jsonparsed_data(url).get('enterpriseValues')
            df = pd.DataFrame.from_records(a)
            try:
                df.date = df.date.apply(lambda x: str(x).replace('-','')).astype('int32')
                df = df.loc[df['date']==int(df.date.max())]
            except:
                pass
            
            df = df.T
            df[Ticker] = df[0]
            del df[0]
            df = pd.DataFrame(df.loc[df.index != 'date'])
            temp_co = temp_co.append(df)
            
            url = ("https://financialmodelingprep.com/api/v3/company-key-metrics/" + str(Ticker))
            a = get_jsonparsed_data(url).get('metrics')
            df = pd.DataFrame.from_records(a)
            try:
                df.date = df.date.apply(lambda x: str(x).replace('-','')).astype('int32')
                df = df.loc[df['date']==int(df.date.max())]
            except:
                pass
            
            df = df.T
            df[Ticker] = df[0]
            del df[0]
            df = pd.DataFrame(df.loc[df.index != 'date'])
            temp_co = temp_co.append(df)
            maindf = pd.concat([maindf, temp_co], axis=1)
            print(maindf.shape)
        
        except:
            continue
        
    try:
        os.remove(str(Tickers[0])+'_all_fin_data.csv')
    except:
        pass
    
    print(maindf.shape)
    return maindf.to_csv(str(Tickers[0])+'_all_fin_data.csv')



In [8]:
import os

def multi_task_fin():
    Tickers = get_jsonparsed_data('https://financialmodelingprep.com/api/v3/company/stock/list')
    Tickers = pd.DataFrame.from_records(Tickers.get('symbolsList'))
    Tickers = list(Tickers['symbol'].values)
    #Tickers = Tickers[:10]
    tn = int(len(Tickers)/4)
    t1 = Tickers[:tn]
    t2 = Tickers[tn:tn*2]
    t3 = Tickers[tn*2:tn*3]
    t4 = Tickers[tn*3:]

    tl = [t1, t2, t3, t4]
    
    # make the Pool of workers
    pool = ThreadPool(8)
    results = pool.map(get_fin_data, tl)
    
    # close the pool and wait for the work to finish
    pool.close()
    pool.join()
    
    import os

    directory = str(os.getcwd())
    old_snap = []
    for file in os.listdir(directory):
        if file.endswith('_all_fin_data.csv'):
            old_snap.append(file)

    big = pd.DataFrame()
    
    for file in old_snap:
        t = pd.read_csv(file)
        t['Ticker'] = t['Unnamed: 0']
        del t['Unnamed: 0']
        t = t.set_index('Ticker')
        big = pd.concat([big, t], axis=1)
    
    print("Done!")
    
    try:
        for file in old_snap():
            os.remove(file)
    except:
        pass
    
    return big

In [9]:
df = multi_task_fin()
df.sample(n=20)

(154, 39)
(154, 39)
(154, 28)
(154, 40)
(154, 1)
(154, 1)
(154, 40)
(154, 1)
(154, 29)
(154, 2)
(154, 2)
(154, 41)(154, 2)

(154, 30)
(154, 3)
(154, 41)
(154, 3)
(154, 3)
(154, 42)
(154, 31)
(154, 4)(154, 42)

(154, 4)
(154, 4)
(154, 43)
(154, 32)
(154, 5)
(154, 5)(154, 44)

(154, 33)
(154, 43)
(154, 6)
(154, 45)
(154, 6)
(154, 44)
(154, 34)
(154, 7)
(154, 46)
(154, 7)
(154, 45)
(154, 35)
(154, 47)
(154, 5)
(154, 8)
(154, 8)
(154, 46)
(154, 48)
(154, 6)
(154, 36)
(154, 9)
(154, 9)
(154, 47)
(154, 10)
(154, 10)
(154, 49)
(154, 37)
(154, 48)
(154, 11)
(154, 11)
(154, 50)
(154, 38)
(154, 7)
(154, 49)
(154, 12)
(154, 12)
(154, 51)
(154, 8)
(154, 39)
(154, 13)
(154, 52)
(154, 13)
(154, 9)
(154, 40)
(154, 50)
(154, 14)
(154, 53)
(154, 14)
(154, 51)(154, 41)

(154, 15)
(154, 54)
(154, 15)
(154, 42)
(154, 16)
(154, 52)
(154, 55)
(154, 16)
(154, 43)
(154, 17)
(154, 56)
(154, 53)
(154, 17)
(154, 10)
(154, 18)
(154, 57)
(154, 54)
(154, 18)
(154, 11)
(154, 44)
(154, 19)
(154, 55)
(154, 58)
(154, 1

(154, 142)
(154, 107)
(154, 189)
(154, 150)
(154, 180)
(154, 143)
(154, 108)
(154, 190)
(154, 151)
(154, 144)
(154, 181)
(154, 109)
(154, 134)
(154, 191)
(154, 152)
(154, 145)
(154, 182)
(154, 192)
(154, 153)(154, 135)

(154, 146)
(154, 183)
(154, 193)
(154, 136)
(154, 147)
(154, 154)
(154, 184)
(154, 110)
(154, 194)
(154, 148)
(154, 137)
(154, 155)
(154, 111)
(154, 195)
(154, 149)
(154, 185)
(154, 156)
(154, 112)
(154, 196)
(154, 150)
(154, 186)
(154, 157)
(154, 138)
(154, 197)
(154, 113)
(154, 151)
(154, 187)
(154, 158)
(154, 139)
(154, 198)
(154, 152)
(154, 188)(154, 159)

(154, 140)
(154, 153)
(154, 199)
(154, 160)(154, 189)

(154, 141)
(154, 200)
(154, 154)
(154, 190)
(154, 161)
(154, 201)
(154, 155)
(154, 162)
(154, 191)
(154, 202)
(154, 114)
(154, 156)
(154, 163)
(154, 192)
(154, 203)
(154, 115)
(154, 157)
(154, 164)
(154, 193)
(154, 204)
(154, 158)
(154, 165)
(154, 116)
(154, 194)
(154, 205)
(154, 142)
(154, 159)
(154, 166)
(154, 195)
(154, 206)
(154, 117)
(154, 196)
(154, 167)

(154, 290)
(154, 277)(154, 330)

(154, 315)
(154, 202)
(154, 291)
(154, 331)(154, 278)

(154, 316)
(154, 203)
(154, 232)
(154, 292)
(154, 332)
(154, 279)
(154, 317)
(154, 233)
(154, 204)
(154, 293)
(154, 280)
(154, 318)
(154, 205)
(154, 333)
(154, 294)
(154, 281)
(154, 319)
(154, 334)
(154, 295)
(154, 282)(154, 206)

(154, 234)
(154, 335)
(154, 320)
(154, 296)
(154, 207)
(154, 283)
(154, 336)
(154, 297)
(154, 321)
(154, 208)
(154, 284)
(154, 235)
(154, 298)
(154, 337)
(154, 322)
(154, 285)
(154, 209)
(154, 236)
(154, 299)
(154, 1)
(154, 338)
(154, 323)
(154, 286)
(154, 300)
(154, 237)
(154, 339)
(154, 324)
(154, 287)
(154, 301)
(154, 238)
(154, 288)
(154, 325)
(154, 302)
(154, 239)
(154, 210)
(154, 340)
(154, 326)
(154, 289)
(154, 303)
(154, 211)
(154, 341)
(154, 290)
(154, 240)
(154, 304)
(154, 327)
(154, 212)
(154, 342)
(154, 291)
(154, 241)
(154, 328)
(154, 305)
(154, 213)
(154, 343)
(154, 292)
(154, 242)
(154, 306)
(154, 329)
(154, 344)
(154, 293)
(154, 243)
(154, 307)
(154, 330)
(

(154, 460)
(154, 419)
(154, 293)
(154, 401)
(154, 435)
(154, 332)
(154, 461)
(154, 36)
(154, 420)
(154, 294)
(154, 436)
(154, 402)
(154, 333)
(154, 462)
(154, 295)
(154, 421)
(154, 437)
(154, 403)
(154, 334)
(154, 463)
(154, 296)
(154, 422)
(154, 438)
(154, 404)
(154, 335)
(154, 464)
(154, 297)
(154, 423)
(154, 439)
(154, 405)
(154, 336)
(154, 298)
(154, 424)
(154, 440)
(154, 406)
(154, 465)
(154, 337)
(154, 299)
(154, 425)
(154, 407)
(154, 466)
(154, 426)
(154, 338)
(154, 467)
(154, 28)
(154, 300)
(154, 427)
(154, 408)
(154, 441)
(154, 339)
(154, 468)
(154, 301)
(154, 428)
(154, 409)
(154, 442)
(154, 340)
(154, 469)
(154, 302)
(154, 429)
(154, 410)
(154, 443)
(154, 303)
(154, 470)
(154, 341)
(154, 29)
(154, 430)
(154, 411)
(154, 37)
(154, 304)
(154, 471)
(154, 342)
(154, 444)
(154, 431)
(154, 38)
(154, 412)
(154, 343)
(154, 305)
(154, 472)
(154, 445)
(154, 30)
(154, 432)
(154, 413)
(154, 344)(154, 306)

(154, 473)
(154, 446)
(154, 433)
(154, 414)
(154, 345)
(154, 474)
(154, 307)
(154,

(154, 518)
(154, 549)
(154, 450)
(154, 414)
(154, 550)
(154, 550)(154, 451)

(154, 415)
(154, 588)
(154, 551)
(154, 519)
(154, 551)(154, 416)

(154, 589)
(154, 552)
(154, 452)
(154, 520)
(154, 417)
(154, 552)
(154, 590)
(154, 553)
(154, 453)
(154, 418)
(154, 521)
(154, 553)
(154, 591)
(154, 554)
(154, 454)
(154, 419)
(154, 522)
(154, 554)
(154, 592)
(154, 555)
(154, 455)
(154, 420)
(154, 523)
(154, 555)
(154, 593)
(154, 456)
(154, 556)
(154, 421)
(154, 524)
(154, 556)
(154, 594)
(154, 457)
(154, 557)
(154, 422)
(154, 525)
(154, 557)
(154, 595)
(154, 558)
(154, 423)
(154, 458)
(154, 526)
(154, 558)
(154, 596)
(154, 559)
(154, 424)
(154, 459)
(154, 527)
(154, 559)
(154, 597)
(154, 425)
(154, 460)
(154, 528)
(154, 560)
(154, 560)
(154, 598)
(154, 426)
(154, 461)
(154, 529)
(154, 561)
(154, 599)
(154, 427)
(154, 561)
(154, 462)
(154, 562)
(154, 530)
(154, 600)
(154, 428)
(154, 562)
(154, 463)
(154, 563)
(154, 601)
(154, 531)
(154, 563)
(154, 464)
(154, 564)
(154, 602)
(154, 564)
(154, 465)

...

(154, 1792)
(154, 1487)
(154, 1696)
(154, 1793)
(154, 1697)
(154, 1488)(154, 1794)

(154, 1441)
(154, 1698)
(154, 1442)
(154, 1699)
(154, 1795)
(154, 1489)
(154, 1443)
(154, 1700)
(154, 1796)
(154, 1444)
(154, 1701)
(154, 1797)
(154, 1445)
(154, 1702)
(154, 1798)
(154, 1446)
(154, 1490)
(154, 1703)
(154, 1799)
(154, 1704)
(154, 1800)
(154, 1447)
(154, 1705)
(154, 1491)
(154, 1801)
(154, 1706)
(154, 1802)
(154, 1707)
(154, 1803)
(154, 1448)
(154, 1708)
(154, 1804)
(154, 1449)
(154, 1709)
(154, 1805)
(154, 1492)
(154, 1710)
(154, 1806)
(154, 1493)
(154, 1807)
(154, 1711)
(154, 1494)
(154, 1808)
(154, 1450)
(154, 1712)
(154, 1809)
(154, 1495)
(154, 1451)
(154, 1713)
(154, 1810)
(154, 1714)
(154, 1496)
(154, 1811)
(154, 1497)
(154, 1715)
(154, 1812)
(154, 1452)
(154, 1716)
(154, 1498)
(154, 1813)
(154, 1717)
(154, 1453)
(154, 1499)
(154, 1814)
(154, 1718)
(154, 1454)
(154, 1500)
(154, 1815)
(154, 1719)
(154, 1501)
(154, 1816)
(154, 1455)
(154, 1720)
(154, 1817)
(154, 1721)
(154, 1456)
(154

(154, 1953)
(154, 2048)
(154, 1954)
(154, 2049)
(154, 1955)
(154, 2050)
(154, 1956)
(154, 2051)
(154, 1957)
(154, 2052)
(154, 1958)
(154, 2053)
(154, 1959)
(154, 2054)
(154, 1960)
(154, 2055)
(154, 1961)
(154, 2056)
(154, 1962)
(154, 2057)
(154, 1963)
(154, 2058)
(154, 1964)
(154, 2059)
(154, 1965)
(154, 2060)
(154, 1966)
(154, 2061)
(154, 2062)
(154, 1967)
(154, 1968)
(154, 2063)
(154, 1969)
(154, 2064)
(154, 1970)
(154, 2065)
(154, 1971)
(154, 1972)
(154, 2066)
(154, 1973)
(154, 2067)
(154, 1974)
(154, 2068)
(154, 2069)
(154, 1975)
(154, 2070)
(154, 1976)
(154, 2071)
(154, 1977)
(154, 2072)
(154, 1978)
(154, 2073)
(154, 1979)
(154, 2074)
(154, 1980)
(154, 2075)
(154, 1981)
(154, 2076)
(154, 1982)
(154, 2077)
(154, 1983)
(154, 2078)
(154, 1984)
(154, 2079)
(154, 1985)
(154, 2080)
(154, 1986)
(154, 1987)
(154, 2081)
(154, 1988)
(154, 2082)
(154, 1989)
(154, 2083)
(154, 1990)
(154, 2084)
(154, 1991)
(154, 2085)
(154, 1992)
(154, 2086)
(154, 1993)
(154, 2087)
(154, 1994)
(154, 2088)
(154

(154, 2388)
(154, 2296)
(154, 2389)
(154, 2297)
(154, 2390)
(154, 2391)
(154, 2298)
(154, 2392)
(154, 2299)
(154, 2393)
(154, 2300)
(154, 2394)
(154, 2301)
(154, 2395)
(154, 2302)
(154, 2396)
(154, 2303)
(154, 2397)
(154, 2304)
(154, 2398)
(154, 2305)
(154, 2399)
(154, 2306)
(154, 2400)
(154, 2307)
(154, 2401)
(154, 2308)
(154, 2309)
(154, 2402)
(154, 2403)
(154, 2310)
(154, 2404)
(154, 2311)
(154, 2405)
(154, 2312)
(154, 2406)
(154, 2313)
(154, 2407)
(154, 2408)
(154, 2314)
(154, 2315)
(154, 2409)
(154, 2316)
(154, 2410)
(154, 2317)
(154, 2411)
(154, 2318)
(154, 2412)
(154, 2319)
(154, 2413)
(154, 2320)
(154, 2414)
(154, 2321)
(154, 2415)
(154, 2322)
(154, 2416)
(154, 2323)
(154, 2417)
(154, 2324)
(154, 2418)
(154, 2325)
(154, 2419)
(154, 2326)
(154, 2420)
(154, 2327)
(154, 2421)
(154, 2328)
(154, 2422)
(154, 2329)
(154, 2423)
(154, 2330)
(154, 2424)
(154, 2331)
(154, 2425)
(154, 2332)
(154, 2426)
(154, 2333)
(154, 2427)
(154, 2334)
(154, 2428)
(154, 2335)
(154, 2429)
(154, 2336)
(154

(154, 2723)
(154, 2644)
(154, 2724)
(154, 2645)
(154, 2646)
(154, 2725)
(154, 2726)
(154, 2647)
(154, 2727)
(154, 2728)
(154, 2648)
(154, 2649)
(154, 2729)
(154, 2650)
(154, 2730)
(154, 2731)
(154, 2651)
(154, 2732)
(154, 2652)
(154, 2733)
(154, 2653)
(154, 2734)
(154, 2654)
(154, 2655)
(154, 2735)
(154, 2656)
(154, 2736)
(154, 2657)
(154, 2737)
(154, 2658)
(154, 2738)
(154, 2659)
(154, 2739)
(154, 2660)
(154, 2740)
(154, 2661)
(154, 2741)
(154, 2662)
(154, 2742)
(154, 2663)
(154, 2743)
(154, 2664)
(154, 2744)
(154, 2665)
(154, 2745)
(154, 2666)
(154, 2746)
(154, 2667)
(154, 2747)
(154, 2668)
(154, 2748)
(154, 2669)
(154, 2749)
(154, 2670)
(154, 2750)
(154, 2671)
(154, 2751)
(154, 2672)
(154, 2673)
(154, 2674)
(154, 2752)
(154, 2675)
(154, 2753)
(154, 2676)
(154, 2754)
(154, 2677)
(154, 2755)
(154, 2678)
(154, 2756)
(154, 2679)
(154, 2757)
(154, 2680)
(154, 2758)
(154, 2681)
(154, 2759)
(154, 2682)
(154, 2760)
(154, 2683)
(154, 2761)
(154, 2684)
(154, 2762)
(154, 2685)
(154, 2763)
(154

,CMCSA,KMI,INTC,MU,GE,BAC,AAPL,MSFT,SIRI,HPQ,...,THMO,SNCA,PHGE,ELA,DPHCW,AGBAR,OPGNW,HHHHR,TKKSR,TOTAR
Ticker,,,,,,,,,,,,,,,,,,,,,
Working Capital,-5.755E9,-1.862E9,8.929E9,1.0113E10,1.6191E10,6.85753E11,5.7101E10,1.06132E11,-2.324445E9,-5.116E9,...,3176000.0,4422124.0,7.8307E7,4131668.0,930355.0,400403.0,1728762.0,-209466.0,-748591.0,-940014.0
Earnings Yield,0.06536701592969271,0.04986157152744891,0.08208829054293985,0.1311494072827912,-0.3599,0.08724722874747523,0.05724981145517573,0.041348934596860114,0.04170018671439788,0.11191651709549655,...,-0.6287423626604962,-3.276412005061829,-0.3698582561715804,-0.012125203228038792,NaN,0.5147606712667018,-5.0641732375348205,0.21452342869236724,2.0501452393470108,-1.0689118924950733
R&D to Revenue,0.0,0.04466651525475055,0.08545820885152505,0.035717337434845765,0.29784154915100935,0.6540467570718537,0.07012614634821313,0.1835461646654959,0.1457126805589347,0.09136088229287222,...,0.6156970951176516,297.8847602962193,NaN,0.21407579000628826,NaN,NaN,2.205777202057729,NaN,NaN,NaN
Total debt,1.11743e+11,34392000000.0,29001000000.0,5851000000.0,89227000000.0,2.44797e+11,1.02067e+11,72178000000.0,6887983000.0,5137000000.0,...,3461000.0,180912.0,1231000.0,97837.0,0.0,543193.0,1346870.0,965641.0,0.0,920000.0
Investing Cash flow,-50854000000.0,-1714000000.0,-14405000000.0,-10085000000.0,8939000000.0,-80630000000.0,45896000000.0,-15888000000.0,-379276000.0,-438000000.0,...,-182000.0,-6018.0,19740000.0,-191132.0,-279034000.0,-46000015.0,-2502576.0,-57500000.0,0.0,314336.0
Shareholders Equity per Share,15.622382198952879,14.903710247349823,17.54675118858954,32.20915619389587,3.5618,28.19977636973537,19.59533408953202,13.336374299491725,-0.4072145782433967,-0.7874587458745874,...,0.1364629785837971,2.3327176151820717,15.038847149926921,0.4469448161857488,NaN,2.6126555966729406,2.6552263257921296,2.196346247697681,0.7191786348755254,2.3742258838054084
Invested Capital,164387000000.0,NaN,NaN,39628000000.0,337396000000.0,NaN,292001000000.0,111907000000.0,7413000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation & Amortization,10676000000.0,2494000000.0,10826000000.0,5473000000.0,5595000000.0,-337000000.0,12547000000.0,11682000000.0,307241000.0,744000000.0,...,3154000.0,143859.0,318000.0,286747.0,0.0,0.0,921379.0,0.0,0.0,0.0
Deposit Liabilities,0.0,0.0,0.0,0.0,0.0,1.434803e+12,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df.shape

(154, 5412)

In [11]:
df = df.T

try:
    os.remove('all_company_data.csv')
except:
    pass
df.to_csv('all_company_data.csv')

In [13]:
df

(154, 2935)


Ticker,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Average Payables,Average Inventory,Days Sales Outstanding,Days Payables Outstanding,Days of Inventory on Hand,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share
CMCSA,94507000000.0,0.118082010269,0.0,0.0,0.0,0.0,7036000000.0,19009000000.0,3542000000.0,15242000000.0,...,7.71E9,0.0,42.88528892039743,45.28512167333703,0.0,8.511077089337176,8.060042382858489,0.0,0.16381103989499113,2.1321989528795813
KMI,13209000000.0,-0.0661057692308,3263000000.0,9946000000.0,0.0,590000000.0,-1208000000.0,4873000000.0,0.0,3165000000.0,...,1.1255E9,3.78E8,37.856764327352565,102.24026969046889,41.50015323322096,9.641605839416059,3.5700218818380742,8.795148247978437,0.0649042736055954,1.0026501766784452
INTC,71965000000.0,0.0157661472448,29825000000.0,42140000000.0,13362000000.0,6150000000.0,20105000000.0,22035000000.0,489000000.0,24058000000.0,...,3.976E9,7.9985E9,38.84575835475578,50.518692372171,107.00955574182733,9.396135265700483,7.2250484496124034,3.4109103385178408,0.27157308009909165,3.6705908988000906
MU,23406000000.0,-0.229837780922,12704000000.0,10702000000.0,2441000000.0,836000000.0,3277000000.0,7376000000.0,128000000.0,7048000000.0,...,4.6185E9,4.3565E9,0.0,132.91010705289673,147.0458123425693,8.42548596112311,2.7462170341547774,2.4822196170379054,0.17594269947883281,8.779174147217235
GE,95214000000.0,-0.217086708054,70029000000.0,25185000000.0,0.0,13949000000.0,14407000000.0,1149000000.0,0.0,1149000000.0,...,1.7153E10,1.9271E10,59.64732968794967,49.91385839452138,56.077069032870135,6.119301600080507,7.312598379292252,6.508899382491827,-0.7215712856266744,-0.4066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGBAR,0.0,0.0,0.0,0.0,0.0,10000.0,338649.0,167472.0,0.0,167472.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.03349439330112134,0.0
OPGNW,3498614.0,0.18745738309,911565.0,2587049.0,5121168.0,7717163.0,12848190.0,-12271245.0,187549.0,-12446458.0,...,1339893.0,508388.5,59.23803397574011,422.8472736447757,189.40607636317762,6.16158193483395,0.8631958221081688,1.9270765067754687,-2.89901589415604,0.0196828699165522
HHHHR,0.0,0.0,936745.0,0.0,0.0,936745.0,936745.0,-936745.0,0.0,0.0,...,NaN,NaN,NaN,86.46620478358571,0.0,NaN,4.221302425769122,NaN,0.030278587888564846,0.0
TKKSR,0.0,0.0,0.0,0.0,0.0,0.0,2351687.0,-2351687.0,0.0,3278278.0,...,302100.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.6556554688689062,0.0
